<a href="https://colab.research.google.com/github/siti-alawiyah/ibresultprediction/blob/main/RandomForestReg-1.02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
# ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,make_scorer, r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn import metrics
import pickle

import statsmodels.api as sm
from scipy import stats

In [2]:
# links to dataset
URL1 = "https://raw.githubusercontent.com/siti-alawiyah/ibresultprediction/main/data/2020IB.csv"
URL2 = "https://raw.githubusercontent.com/siti-alawiyah/ibresultprediction/main/data/2019IB.csv"
URL3 = "https://raw.githubusercontent.com/siti-alawiyah/ibresultprediction/main/data/2018IB.csv"   
URL4 = "https://raw.githubusercontent.com/siti-alawiyah/ibresultprediction/main/data/2017IB.csv"      

In [3]:
# read the url for the datasets
df20 = pd.read_csv(URL1)
df19 = pd.read_csv(URL2)
df18 = pd.read_csv(URL3)
df17 = pd.read_csv(URL4)

In [4]:
# getting the columns prior for modelling
col_name = ["Ma Std","Ma Teacher","Ma ATL","Ma Compl. Of Work","Ma Sub Achievement","Predicted Grade","Actual Grade","Scaled Total"]

df20 = df20[col_name]
df19 = df19[col_name]
df18 = df18[col_name]
df17 = df17[col_name]

In [5]:
# combine the dataframes into 1 before modelling
frames = [df20,df19,df18,df17]
df = pd.concat(frames)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 681 entries, 0 to 177
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ma Std              657 non-null    object 
 1   Ma Teacher          657 non-null    object 
 2   Ma ATL              657 non-null    float64
 3   Ma Compl. Of Work   654 non-null    float64
 4   Ma Sub Achievement  657 non-null    float64
 5   Predicted Grade     654 non-null    float64
 6   Actual Grade        652 non-null    float64
 7   Scaled Total        653 non-null    float64
dtypes: float64(6), object(2)
memory usage: 47.9+ KB


In [7]:
df.isnull().sum()

Ma Std                24
Ma Teacher            24
Ma ATL                24
Ma Compl. Of Work     27
Ma Sub Achievement    24
Predicted Grade       27
Actual Grade          29
Scaled Total          28
dtype: int64

In [8]:
#drop rows that have null values
df.dropna(axis=0, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 650 entries, 0 to 177
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ma Std              650 non-null    object 
 1   Ma Teacher          650 non-null    object 
 2   Ma ATL              650 non-null    float64
 3   Ma Compl. Of Work   650 non-null    float64
 4   Ma Sub Achievement  650 non-null    float64
 5   Predicted Grade     650 non-null    float64
 6   Actual Grade        650 non-null    float64
 7   Scaled Total        650 non-null    float64
dtypes: float64(6), object(2)
memory usage: 45.7+ KB


In [10]:
# dummify Ma Std and Ma Teacher
col= ['Ma Std','Ma Teacher']

# Creaing dummies 
df = pd.get_dummies(columns=col, data=df,drop_first=True)

In [11]:
# create train test split 
X = df.drop(['Predicted Grade','Actual Grade'],axis=1)
y = df['Actual Grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [12]:
pipe  = Pipeline([('rf', RandomForestRegressor(random_state=42))])

In [14]:
pipe_params = {
    'rf__n_jobs': [-1],
    'rf__n_estimators': [10,50,100,150,200,250],
    'rf__max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'rf__random_state': [42]
    }

In [15]:
scorers = {'RMSE': make_scorer(mean_squared_error), 'R2': make_scorer(r2_score)}

In [16]:
# instantiate GridSeachCV
gs = GridSearchCV(pipe, param_grid = pipe_params, cv = 3, scoring = scorers, n_jobs = -1, refit = 'R2')

In [17]:
# fit grid search to training data
gs.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('rf',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'rf__n_estimators': [10, 50, 100, 150, 200, 250],
                         'rf__n_jobs': [-1], 'rf__random_state': [42]},
             refit='R2',
             scoring={'R2': make_scorer(r2_score),
                      'RMSE': make_scorer(mean_squared_error)})

In [18]:
#print the best score for CV = 3
print(gs.best_score_)

0.9399201158079015


In [19]:
#model score on training set
gs.score(X_train, y_train)

0.9881054416216698

In [20]:
#score model on test set
gs.score(X_test, y_test)

0.9603395819791885

In [21]:
#get the best params
gs.best_params_

{'rf__max_depth': 7,
 'rf__n_estimators': 100,
 'rf__n_jobs': -1,
 'rf__random_state': 42}

In [22]:
#RandomForest AUC/Accuracy Score
rf_r2_train = gs.score(X_train,y_train)
rf_r2_test = gs.score(X_test,y_test)
rf_rmse_train = mean_squared_error(gs.predict(X_train),y_train)
rf_rmse_test = mean_squared_error(gs.predict(X_test),y_test)

In [24]:
print(f'RandomForestRegressor R2 train score {rf_r2_train}')
print(f'RandomForestRegressor R2 test score {rf_r2_test}')
print(f'RandomForestRegressor RMSE train score {rf_rmse_train}')
print(f'RandomForestRegressor RMSE test score {rf_rmse_test}')

RandomForestRegressor R2 train score 0.9881054416216698
RandomForestRegressor R2 test score 0.9603395819791885
RandomForestRegressor RMSE train score 0.015880133164691262
RandomForestRegressor RMSE test score 0.05189387082728364
